Este script toma un chingo de imagenes de una carpeta y las convierte en un objeto 3d

In [ ]:
import os
import subprocess

# === CONFIGURA ESTO ===
frames_dir = "frames_por_tramos/tramo_1"  # Carpeta con los frames
project_name = "modelo_estadio"           # Nombre del proyecto/carpeta
output_model_name = "estadio_modelo.ply"  # Nombre final del archivo .ply

# === DIRECTORIOS INTERNOS ===
base_dir = os.path.join("proyectos_colmap", project_name)
db_path = os.path.join(base_dir, "colmap.db")
sparse_dir = os.path.join(base_dir, "sparse")
dense_dir = os.path.join(base_dir, "dense")
os.makedirs(base_dir, exist_ok=True)
os.makedirs(sparse_dir, exist_ok=True)
os.makedirs(dense_dir, exist_ok=True)

# === 1. EXTRAER FEATURES ===
print("🔍 Extrayendo features...")
subprocess.run([
    "colmap", "feature_extractor",
    "--database_path", db_path,
    "--image_path", frames_dir
])

# === 2. MATCHING ===
print("🔗 Matching de imágenes...")
subprocess.run([
    "colmap", "exhaustive_matcher",
    "--database_path", db_path
])

# === 3. SPARSE RECONSTRUCTION ===
print("📸 Reconstrucción sparse...")
subprocess.run([
    "colmap", "mapper",
    "--database_path", db_path,
    "--image_path", frames_dir,
    "--output_path", sparse_dir
])

# === 4. UNDISTORT IMAGES ===
print("🛠️ Undistort para reconstrucción densa...")
subprocess.run([
    "colmap", "image_undistorter",
    "--image_path", frames_dir,
    "--input_path", os.path.join(sparse_dir, "0"),
    "--output_path", dense_dir,
    "--output_type", "COLMAP",
    "--max_image_size", "2000"
])

# === 5. DENSE RECONSTRUCTION ===
print("🔬 Reconstrucción densa...")
subprocess.run([
    "colmap", "patch_match_stereo",
    "--workspace_path", dense_dir,
    "--workspace_format", "COLMAP",
    "--PatchMatchStereo.geom_consistency", "true"
])

# === 6. FUSION ===
print("💥 Fusionando nube de puntos...")
fused_ply = os.path.join(dense_dir, "fused_temp.ply")
subprocess.run([
    "colmap", "stereo_fusion",
    "--workspace_path", dense_dir,
    "--workspace_format", "COLMAP",
    "--input_type", "geometric",
    "--output_path", fused_ply
])

# === 7. MESHER ===
print("🧱 Generando malla...")
final_model_path = os.path.join(base_dir, output_model_name)
subprocess.run([
    "colmap", "poisson_mesher",
    "--input_path", fused_ply,
    "--output_path", final_model_path,
    "--depth", "8"
])

print(f"✅ Modelo generado en: {final_model_path}")
